# Abstract Polynomials

This is an experimental notebook.

## Ordinary Polynomials

Rolling my own polynomials just to see how difficult/useful this might be...

In [47]:
import functools as fnc


def power(x, n):
    result = 1
    for _ in range(n):
        result = result * x
    return result


class Term:
    """Represents a term of a polynomial.  Here's an example polynomial with four
    terms, separated by spaces: -2 -4x +7x^2 -3x^4
    """
    
    def __init__(self, coefficient, order):
        self.__coefficient = coefficient
        self.__order = order
        self.__varname = "x"
        
    def __repr__(self):
        return f"Term({self.__coefficient},{self.__order})"
    
    def __str__(self):
        if self.__order == 0:
            return f"{self.__coefficient}"
        elif self.__order == 1:
            return f"{self.__coefficient}{self.__varname}"
        else:
            return f"{self.__coefficient}{self.__varname}^{self.__order}"
        
    def __call__(self, x):
        return self.__coefficient * power(x, self.__order)
    
    def __add__(self, other):
        if self.__order == other.__order:
            return Term(self.__coefficient + other.__coefficient, self.__order)
        else:
            raise ValueError(f"Terms must be of the same order, {self.__order} != {other.__order}")
    
    def __mul__(self, other):
        return Term(self.__coefficient * other.__coefficient,
                    self.__order + other.__order)
    
    def __eq__(self, other):
        return (self.__varname == other.__varname and
                self.__order == other.__order and
                self.__coefficient == other.__coefficient)
    
    @property
    def coefficient(self):
        return self.__coefficient
    
    @property
    def order(self):
        return self.__order
    
    def varname(self, newname=None):
        if newname is not None:
            if isinstance(newname, str):
                self.__varname = newname
            else:
                raise ValueError("Variable name must be a string.")
        return self.__varname
    
    def is_constant(self):
        if self.__order == 0:
            return self.__coefficient
        else:
            return False
        
    def is_linear(self):
        if self.__order == 1:
            return self.__coefficient
        else:
            return False


class Polynomial:
    
    def __init__(self, poly_string, varname):
        self.__terms = sorted(parse_polynomial(poly_string, varname),
                              key=lambda t: t.order)

    def __repr__(self):
        poly_str = ""
        for term in self.__terms:
            poly_str += " " + str(term)
        return poly_str
    
    def __call__(self, x):
        return fnc.reduce(lambda a,b: a + b,
                          map(lambda term: term(x), self.__terms))
    
    def terms(self):
        return self.__terms


def parse_term(term_str, varname):
    """A hacky term string parser.  Returns a Term from the input string."""

    if varname in term_str:
        varpower = varname + "^"
        if varpower in term_str:
            args = list(map(lambda x: int(x), term_str.split(varpower)))
        else:
            args = [int(term_str.split(varname)[0]), 1]
    else:
        args = [int(term_str), 0]

    return Term(args[0], args[1])


def parse_polynomial(poly_str, varname):
    """An extreme hack.  Terms in polynomials have to be separated by a space.
    Example polynomial string where varname is 'x': '-2 -4x +7x^2 -3x^4'
    """
    return [parse_term(term, varname) for term in poly_str.split()]

In [52]:
polystr = "-2 -4x +7x^2 -3x^4"

p = Polynomial(polystr, 'x')
print(p)
p.terms()

 -2 -4x 7x^2 -3x^4


[Term(-2,0), Term(-4,1), Term(7,2), Term(-3,4)]

In [53]:
y = 3
print(-2 - 4*y + 7*y**2 - 3*y**4)

-194


In [54]:
p(3)

-194

In [55]:
polystr = "-2 -4x^1 +7x^2 -3x^4"  # Added '^1' to linear term

p = Polynomial(polystr, 'x')
print(p)
p.terms()

 -2 -4x 7x^2 -3x^4


[Term(-2,0), Term(-4,1), Term(7,2), Term(-3,4)]

In [56]:
polystr = "-2x^0 -4x^1 +7x^2 -3x^4"  # Added 'x^0' to constant term

p = Polynomial(polystr, 'x')
print(p)
p.terms()

 -2 -4x 7x^2 -3x^4


[Term(-2,0), Term(-4,1), Term(7,2), Term(-3,4)]

In [57]:
polystr = "-4x -2 -3x^4 +7x^2"  #  Terms not in order 

p = Polynomial(polystr, 'x')
print(p)
p.terms()

 -2 -4x 7x^2 -3x^4


[Term(-2,0), Term(-4,1), Term(7,2), Term(-3,4)]

## NumPy Polynomials

In [31]:
from numpy.polynomial import Polynomial

poly_coeff = [-2, -4, 7, 0, -3]
poly = Polynomial(poly_coeff)
poly

Polynomial([-2., -4.,  7.,  0., -3.], domain=[-1,  1], window=[-1,  1])

In [32]:
poly(3)

-194.0

In [35]:
print(poly)

-2.0 - 4.0·x¹ + 7.0·x² + 0.0·x³ - 3.0·x⁴


In [37]:
poly1 = Polynomial([1, 1])
poly1

Polynomial([1., 1.], domain=[-1,  1], window=[-1,  1])

In [38]:
poly2 = Polynomial([1, -1])
poly2

Polynomial([ 1., -1.], domain=[-1,  1], window=[-1,  1])

In [41]:
poly1 * poly2

Polynomial([ 1.,  0., -1.], domain=[-1.,  1.], window=[-1.,  1.])